In [1]:
import os
import numpy as np
import pandas as pd

## Import dataset from Mike and Barbara's sentiment

In [34]:
df = pd.read_parquet("https://github.com/Mlad-en/Intro_Data_Science/raw/main/cleaned_data/climate_change_sentiment_and_proportion_talked_about_climate_change.parquet")

# !!!Your code and the sentiment analysis should actually come above. The sentiment analysis code is provided below

In [3]:
## Drop the rows where the text is NaN (so no speech)
# df = df[df['text_split'].notna()]

# from nltk.sentiment import SentimentIntensityAnalyzer
# sia = SentimentIntensityAnalyzer()

# df['sentiment'] = [sia.polarity_scores(speech)['compound'] for speech in df['text_split']]

## Import dataset with continents

In [35]:
df_codes = pd.read_csv('UNSD — Methodology.csv', sep=";")
df_codes = df_codes[['Region Name', 'Sub-region Name', 'ISO-alpha3 Code' ]].rename(columns={'ISO-alpha3 Code':'iso_3'})

## Import dataset with co2 emissions

In [36]:
df_climate = pd.read_csv('co-emissions-per-capita.csv', sep=",", dtype=str)
df_climate = df_climate.rename(columns={'Code':'iso_3', 'Year':'year'})


In [37]:
df_climate['Annual CO₂ emissions (per capita)']=df_climate['Annual CO₂ emissions (per capita)'].str.replace(".", "").astype(float)
df_climate['year'] = df_climate['year'].astype(int)

C:\Users\easto\AppData\Local\Temp\ipykernel_41476\2256849972.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_climate['Annual CO₂ emissions (per capita)']=df_climate['Annual CO₂ emissions (per capita)'].str.replace(".", "").astype(float)


## Merge

In [38]:
df_final = pd.merge(df, df_climate, on=['iso_3', 'year'])

In [39]:
df_final = pd.merge(df_final, df_codes, on=['iso_3'])

## Order

In [40]:
df_final.sort_values(['iso_3', 'year'], ascending=True, inplace=True)

## Add the difference in CO2 and the CO2 of next year, per country

In [41]:
diff = []
for i in sorted(list(set(df_final.iso_3.values))):
    df = df_final[df_final['iso_3'] == i][['iso_3', 'year', 'Annual CO₂ emissions (per capita)']]
    df['diff_co2'] = df['Annual CO₂ emissions (per capita)'].diff()
    for j in df['diff_co2'].values:
        diff.append(j)

In [49]:
y = []
for i in sorted(list(set(df_final.iso_3.values))):
    df = df_final[df_final['iso_3'] == i][['iso_3', 'year', 'Annual CO₂ emissions (per capita)']]
    for j in list(df['year'].values)[:-1]:
        t = df[df['year']==(j+1)]['Annual CO₂ emissions (per capita)'].values
        if len(t) > 0:
            y.append(t[0])
        else:
            y.append(np.nan)
    y.append(np.nan)

In [50]:
df_final['diff_co2'] = diff

In [51]:
df_final['next_year_co2'] = y

## Diff for time spent

In [52]:
diff2 = []
for i in sorted(list(set(df_final.iso_3.values))):
    df = df_final[df_final['iso_3'] == i][['iso_3', 'year', 'amount_of_time_spent_on_climate']]
    df['diff_time_spent'] = df['amount_of_time_spent_on_climate'].diff()
    for j in df['diff_time_spent'].values:
        diff2.append(j)

In [53]:
df_final['diff_time_spent'] = diff2

## The dataset

In [54]:
df_final

,iso_3,year,sentiment,total_amount_paragraphs,amount_climate_change_paragraphs,amount_of_time_spent_on_climate,Entity,Annual CO₂ emissions (per capita),Region Name,Sub-region Name,diff_co2,next_year_co2,diff_time_spent
0,AFG,2013,NaN,37,0,0.000000,Afghanistan,29328325.0,Asia,Southern Asia,NaN,2802986.0,NaN
1,AFG,2014,0.427633,22,3,0.136364,Afghanistan,2802986.0,Asia,Southern Asia,-26525339.0,29007638.0,0.136364
2,AFG,2015,0.718400,19,1,0.052632,Afghanistan,29007638.0,Asia,Southern Asia,26204652.0,2617953.0,-0.083732
3,AFG,2016,NaN,32,0,0.000000,Afghanistan,2617953.0,Asia,Southern Asia,-26389685.0,2768525.0,-0.052632
4,AFG,2017,NaN,18,0,0.000000,Afghanistan,2768525.0,Asia,Southern Asia,150572.0,29487586.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527,ZWE,2017,0.419033,15,3,0.200000,Zimbabwe,65053254.0,Africa,Sub-Saharan Africa,-7828969.0,7836386.0,0.200000
1528,ZWE,2018,0.401900,10,1,0.100000,Zimbabwe,7836386.0,Africa,Sub-Saharan Africa,-57216868.0,72386146.0,-0.100000
1529,ZWE,2019,-0.184433,16,3,0.187500,Zimbabwe,72386146.0,Africa,Sub-Saharan Africa,64549760.0,67697036.0,0.087500
1530,ZWE,2020,NaN,26,0,0.000000,Zimbabwe,67697036.0,Africa,Sub-Saharan Africa,-4689110.0,7062929.0,-0.187500


In [55]:
df_final.to_parquet('FINAL_DATA_assignment1.parquet')